In [ ]:
import pandas as pd

: 

In [ ]:
drug_meta_data = pd.read_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/downloads/Repurposing_Public_24Q2_Extended_Primary_Compound_List.csv')
drug_meta_data2 = pd.read_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/downloads/Repurposing_Public_24Q2_Treatment_Meta_Data.csv')
omics = pd.read_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/downloads/OmicsExpressionProteinCodingGenesTPMLogp1BatchCorrected.csv', index_col=0)
outcomes = pd.read_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/downloads/Repurposing_Public_24Q2_LFC_COLLAPSED.csv')
az_628_correlates = pd.read_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/downloads/AZ-628 (BRDBRD-K05804044-001-18-5) in Rep_all_single_pt associations.csv')
imatinib_correlates = pd.read_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/downloads/IMATINIB (BRDBRD-K92723993-001-17-4) in Rep_all_single_pt associations.csv')

## Two Drugs of Interest

In [ ]:
drug_ids = ["BRD-K05804044-001-18-5", "BRD-K92723993-001-17-4"]
drug_meta_data2[(drug_meta_data2["broad_id"].isin(drug_ids)) & ((drug_meta_data2["name"] == "AZ 628") | (drug_meta_data2["name"] == "Imatinib"))].head()

In [ ]:
drug_meta_data[drug_meta_data["IDs"]=="BRD:BRD-K92723993-001-17-4"]

In [ ]:
drug_meta_data[(drug_meta_data["IDs"]=="BRD:BRD-K05804044-001-18-5") & (drug_meta_data["screen"]=="REP.1M")]

## Outcome Data

In [ ]:
outcomes

In [ ]:
drug_ids = ["BRD-K05804044-001-18-5", "BRD-K92723993-001-17-4"]
filt_outcomes = outcomes[(outcomes["broad_id"].isin(drug_ids)) & (outcomes["screen"]=="REP1M") & ((outcomes["culture"]=="PR500B") | (outcomes["culture"]=="PR500A"))] # & (outcomes["culture"] == "PR500B")]


In [ ]:
filt_outcomes[filt_outcomes["broad_id"]=="BRD-K92723993-001-17-4"]["LFC"].hist(bins=100)

In [ ]:
filt_outcomes[filt_outcomes["broad_id"]=="BRD-K05804044-001-18-5"]["LFC"].hist(bins=100)

In [ ]:
filt_outcomes["cl_id"] = [id.split("::")[0] for id in filt_outcomes["row_id"]]

In [ ]:
filt_outcomes

In [ ]:
filt_outcomes_az_628 = filt_outcomes[filt_outcomes["broad_id"] == "BRD-K05804044-001-18-5"].groupby("cl_id").mean(numeric_only=True)
filt_outcomes_imatinib = filt_outcomes[filt_outcomes["broad_id"] == "BRD-K92723993-001-17-4"].groupby("cl_id").mean(numeric_only=True)
filt_outcomes = pd.DataFrame(filt_outcomes_az_628["LFC"]).join(filt_outcomes_imatinib["LFC"], lsuffix="_az_628", rsuffix="_imatinib")

## Omics Data for Cell Lines

In [ ]:
# Filter for cell lines which were screened for both drugs and are contained in cl_id column from filt_outcomes df
omics = omics[omics.index.isin(filt_outcomes.index)]

# Filter for cell lines which were screened for both drugs and are contained in cl_id column from filt_outcomes df
filt_outcomes = filt_outcomes[filt_outcomes.index.isin(omics.index)]

## Get Top Correlates

In [ ]:
az_628_correlates["Dataset"].unique()

In [ ]:
az_628_genes = az_628_correlates[az_628_correlates["Dataset"] == "Expression Public 24Q2"]["Gene/Compound"]
imatinib_genes = imatinib_correlates[imatinib_correlates["Dataset"] == "Expression Public 24Q2"]["Gene/Compound"]
corr_genes = set(az_628_genes.append(imatinib_genes))

In [ ]:
selected_genes = [col for col in omics.columns if col.split(" ")[0] in corr_genes]
omics_filtered = omics[selected_genes]

In [ ]:
# Remove all genes with very low variance
omics_filtered = omics_filtered.loc[:, omics_filtered.var() > 0.1]

## Combine with outcomes and store

In [ ]:
# Combine omics and outcomes
data = filt_outcomes.join(omics_filtered)

In [ ]:
# Normalize data
import numpy as np
data = data - np.min(data, axis=0) / (np.max(data, axis=0) - np.min(data, axis=0))

In [ ]:
# drop rows with nans
data = data.dropna()

In [ ]:
data.to_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/real/depmap_drug_screen_2_drugs.csv')

In [ ]:
data_all_features = filt_outcomes.join(omics)

In [ ]:
data_all_features = data_all_features.dropna()

In [ ]:
data_all_features.to_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/real/depmap_drug_screen_2_drugs_all_features.csv')

In [ ]:
import numpy as np
data_all_features_norm = data_all_features - np.min(data_all_features, axis=0) / (np.max(data_all_features, axis=0) - np.min(data_all_features, axis=0))
data_all_features.to_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/real/depmap_drug_screen_2_drugs_all_features_norm.csv')